In [1]:
%load_ext autoreload
%autoreload 2

import nest_asyncio
nest_asyncio.apply()

import asyncio
import numpy as np

from teeport import Teeport

### Optimizer integration test

In this test, we'll integrate a mockup algorithm to Teeport and use it later to optimize a mockup problem.

#### Integrate a mockup algorithm to Teeport

Firstly create a new Teeport adapter:

In [2]:
teeport = Teeport('ws://localhost:8080/')

Let's create an optimize function which will be integrated to Teeport later:

In [3]:
def optimize(evaluate, configs=None):
    for i in range(10):
        X_next = np.random.rand(30, 10) / (i + 1)  # 30 points, 10 dimension decision space
        Y = evaluate(X_next)
        print(f'gen {i + 1}: {np.min(Y)}')

Note that the configs argument is needed, but it could be just None. This behavior can be optimized if needed.

Now integrate it by using the `run_optimizer` API:

In [4]:
teeport.run_optimizer(optimize)

True

Get the optimizer id by calling the `status` API: 

In [5]:
teeport.status()

General
uri             : ws://localhost:8080/
name            : t0
topology        : t0      

Optimizer
uri             : ws://localhost:8080/
algorithm id    : None
name            : whispering-plants
configs         : null
private         : False
local           : True
socket          : connected
socket id       : R0oFd0nBZ
optimize        : set
main task       : running
opt task        : not running
eval task       : not running


The `socket id` property is the optimizer id we'll need later. You can now copy it to your clipboard.

#### Use the optimizer to optimize a mockup problem

Assume we're on some other computer, so we'll need to initialize a new Teeport adapter:

In [6]:
teeport2 = Teeport('ws://localhost:8080/')

Get the optimizer through the `use_optimizer` API with the id we just copied:

In [7]:
optimize = teeport2.use_optimizer('R0oFd0nBZ')

Now define a mockup evaluate function:

In [8]:
import time

def evaluate(X, configs=None):
    time.sleep(1)  # emulate the calculation time
    Y = np.linalg.norm(X, axis=1).reshape(-1, 1)
    return Y

Note that the configs argument is needed here, too. We also need to reshape Y to a 2D array, this is a requirement of Teeport. The mockup defined above just returns the norm of each point.

Test the mockup evaluate function:

In [9]:
evaluate(np.ones((30, 10)))

array([[3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766]])

Now let's optimize it:

In [10]:
optimize(evaluate)

gen 1: 1.3794647415613954
gen 2: 0.6646163527265604
gen 3: 0.42463106234007697
gen 4: 0.2916153277966618
gen 5: 0.27916231870650704
gen 6: 0.22033118618652037
gen 7: 0.12691277521534278
gen 8: 0.17923090919302187
gen 9: 0.14660139020304847
gen 10: 0.12432845388968225
optimizer: optimization finished
optimizer: already stopped
evaluator: stopped


#### Clean up

Let's do some cleaning by calling the `reset` API 

In [11]:
teeport.reset()

teeport: not linked yet
optimizer: stopped


In [12]:
teeport2.reset()

teeport: not linked yet
optimizer: already stopped
evaluator: already stopped


This will disconnect all the websockets that connected to the Teeport backend service.

### Evaluator integration test

In this test, we'll integrate a mockup problem to Teeport and use it later to test a mockup algorithm.

#### Integrate a mockup evaluator to Teeport

Firstly create a new Teeport adapter:

In [13]:
teeport = Teeport('ws://localhost:8080/')

Let's create an optimize function which will be integrated to Teeport later:

In [14]:
import time

def evaluate(X, configs=None):
    time.sleep(1)  # emulate the calculation time
    Y = np.linalg.norm(X, axis=1).reshape(-1, 1)
    return Y

Note that the configs argument is needed, but it could be just None. This behavior can be optimized if needed.

Now integrate it by using the `run_evalautor` API:

In [15]:
teeport.run_evaluator(evaluate)

True

Get the evaluator id by calling the `status` API: 

In [16]:
teeport.status()

General
uri             : ws://localhost:8080/
name            : t0
topology        : t0      

Evaluator
uri             : ws://localhost:8080/
problem id      : None
name            : absent-bell
configs         : null
private         : False
local           : True
socket          : connected
socket id       : eC0C15X9f
evaluate        : set
main task       : running


The `socket id` property is the evaluator id we'll need later. You can now copy it to your clipboard.

#### Use the evaluator as a local function

Assume we're on some other computer, so we'll need to initialize a new Teeport adapter:

In [17]:
teeport2 = Teeport('ws://localhost:8080/')

Get the evaluator through the `use_evaluator` API with the id we just copied:

In [18]:
evaluate = teeport2.use_evaluator('eC0C15X9f')

Now assume that you yourself are the optimizer:) Do whatever you want with the evaluator! 

In [19]:
evaluate(np.ones((30, 10)))

array([[3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766],
       [3.16227766]])

In [20]:
evaluate(np.zeros((30, 10)))

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

#### Clean up

Let's do some cleaning by calling the `reset` API 

In [21]:
teeport.reset()

teeport: not linked yet
evaluator: stopped


In [22]:
teeport2.reset()

evaluator: already stopped
optimizer: stopped
optimizer: optimization cancelled
teeport: not linked yet


This will disconnect all the websockets that connected to the Teeport backend service.